In [1]:
!pip install -U transformers datasets evaluate accelerate


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")
dataset


c:\Users\chong\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
# In thử 1 mẫu dữ liệu đầu tiên
print(dataset["train"][0])


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [2]:
from transformers import AutoTokenizer

# Sử dụng tokenizer của mô hình DistilBERT (bản nhẹ của BERT)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Định nghĩa hàm token hóa (chuyển văn bản thành các mã token)
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)


In [3]:
# Áp dụng tokenization cho toàn bộ tập dữ liệu
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# Hiển thị cấu trúc dữ liệu sau khi token hóa
tokenized_datasets


Map: 100%|██████████| 25000/25000 [00:06<00:00, 3889.76 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [4]:
from transformers import AutoModelForSequenceClassification

# Load mô hình DistilBERT với 2 nhãn (positive / negative)
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

# Tạo đối tượng TrainingArguments để định nghĩa tham số huấn luyện
training_args = TrainingArguments(
    output_dir="test_trainer",         
    eval_strategy="epoch",             # Đánh giá sau mỗi epoch
    per_device_train_batch_size=8,     
    per_device_eval_batch_size=8,     
    num_train_epochs=2,                # Số epoch huấn luyện
    weight_decay=0.01,                 # Hệ số regularization (tránh overfitting)
)


In [6]:
from transformers import Trainer
import numpy as np
import evaluate

# Chọn thước đo đánh giá accuracy (độ chính xác)
metric = evaluate.load("accuracy")

# Hàm tính độ chính xác sau mỗi lần đánh giá
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Chọn nhãn có xác suất cao nhất
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Khởi tạo Trainer - tự động xử lý train + eval + logging
trainer = Trainer(
    model=model,                      
    args=training_args,               # tham số huấn luyện
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(2000)),  # chọn 2000 mẫu train
    eval_dataset=tokenized_datasets["test"].select(range(1000)),                     # chọn 1000 mẫu test
    compute_metrics=compute_metrics,  # hàm đánh giá
)


In [7]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.457970,0.826000
2,0.291500,0.423954,0.896000


TrainOutput(global_step=500, training_loss=0.2915445556640625, metrics={'train_runtime': 202.4148, 'train_samples_per_second': 19.761, 'train_steps_per_second': 2.47, 'total_flos': 529869594624000.0, 'train_loss': 0.2915445556640625, 'epoch': 2.0})

In [9]:
import torch

# Câu ví dụ cần dự đoán cảm xúc
text = "This movie was absolutely wonderful!"

# Xác định thiết bị đang dùng (GPU nếu có, ngược lại dùng CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Đưa mô hình lên thiết bị tương ứng
model.to(device)

# Tokenize câu đầu vào và đưa tensor lên cùng thiết bị
inputs = tokenizer(text, return_tensors="pt").to(device)

# Cho mô hình dự đoán (không tính gradient để tiết kiệm tài nguyên)
with torch.no_grad():
    outputs = model(**inputs)

# Lấy nhãn có xác suất cao nhất
pred = outputs.logits.argmax(-1).item()

# In kết quả dự đoán
label = "Positive" if pred == 1 else "Negative"
print(f"Prediction: {label}")


Prediction: Positive
